# Импорт библиотек и создание класса CustomDataset для загрузки данных и их преобразования в изображения

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch.utils.data as data
import torchvision.transforms.v2 as tfs


class CustomDataset(data.Dataset):
    def __init__(self, path, train=True, transform=None):
        self.data = pd.read_csv(path)
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]  # Получаем строку с индексом idx
        label = row.iloc[0] if self.train else None # Целевая переменная
        pixels = row.iloc[1:].values.astype(np.uint8).reshape(50, 50)  # Пиксели изображения
        img = Image.fromarray(pixels)  # Создаем изображение из пикселей

        if self.transform:
            img = self.transform(img)

        if self.train:
            return img, label
        else:
            return img

# Создание модели нейронной сети и загрузка весов 

In [5]:
# Трансформация данных
transform = tfs.Compose([
    tfs.ToTensor(),  # Преобразуем изображение в тензор
    tfs.Normalize(mean=[0.5], std=[0.5]), # Нормализуем
    tfs.ToDtype(torch.float32, scale=True)])  # Приводим к типу float32

model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),  

    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),    

    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Flatten(),

    nn.Linear(128 * 6 * 6, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 37)
)

# ЗАГРУЖАЕМ СОХРАНЁННЫЕ ВЕСА
model.load_state_dict(torch.load('model.pth'))
model.eval()  # Переводим в режим оценки


C:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Flatten(start_dim=1, end_dim=-1)
  (13): Linear(in_features=4608, out_features=256, bias=True)
  (14): ReLU()
  (15): Dropout(p=0.4, inplace=False)
  (16): Linear(in_features=256, out_features=128, bias=True)


# Загрузка данных и обучение модели

In [6]:
#Тестирование модели
d_test = CustomDataset('testnn.csv', train=True, transform=transform)
test_data = data.DataLoader(d_test, batch_size=32, shuffle=False)

correct = 0
total = 0
test_tqdm = tqdm(test_data, leave=True)

with torch.no_grad():
    for x_test, y_test in test_tqdm:
        p = model(x_test)
        _, predicted = torch.max(p, 1)  
        total += y_test.size(0)
        correct += (predicted == y_test).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')

model.eval()
test_dataset = CustomDataset('test.csv', train=False, transform=transform)
test_dataloader = data.DataLoader(test_dataset, batch_size=32, shuffle=False)

correct_test = 0
total_test = 0
predictions = []
test_tqdm = tqdm(test_dataloader, leave=True)

with torch.no_grad():
    for x_test in test_tqdm:
        p = model(x_test)
        _, predicted = torch.max(p, 1)
        predictions.extend(predicted)

print(predictions)
predictions = [p.item() for p in predictions]
print(predictions)

100%|██████████| 502/502 [00:16<00:00, 30.40it/s]


Accuracy: 92.00%


100%|██████████| 279/279 [00:18<00:00, 14.73it/s]


[tensor(21), tensor(11), tensor(21), tensor(31), tensor(19), tensor(34), tensor(29), tensor(19), tensor(25), tensor(10), tensor(29), tensor(11), tensor(11), tensor(36), tensor(17), tensor(32), tensor(9), tensor(9), tensor(36), tensor(28), tensor(12), tensor(6), tensor(26), tensor(33), tensor(4), tensor(5), tensor(4), tensor(9), tensor(10), tensor(16), tensor(25), tensor(14), tensor(17), tensor(31), tensor(24), tensor(10), tensor(15), tensor(35), tensor(27), tensor(6), tensor(13), tensor(29), tensor(14), tensor(15), tensor(22), tensor(29), tensor(14), tensor(14), tensor(9), tensor(23), tensor(9), tensor(10), tensor(5), tensor(27), tensor(32), tensor(32), tensor(10), tensor(5), tensor(2), tensor(10), tensor(9), tensor(9), tensor(4), tensor(10), tensor(18), tensor(14), tensor(36), tensor(1), tensor(16), tensor(16), tensor(18), tensor(32), tensor(13), tensor(23), tensor(17), tensor(36), tensor(29), tensor(36), tensor(28), tensor(33), tensor(10), tensor(4), tensor(30), tensor(30), tensor(16